In [56]:
gold_test_path = "dev_aspects.txt"
algs = ['lbfgs', 'l2sgd', 'ap', 'pa', 'arow']
pred_test_path = f"pred_asp_{algs[0]}_dict.txt"

# Оценка 1: accuracy по выделению упоминаний с категориями

In [57]:
from collections import defaultdict

In [58]:
gold_aspect_cats = {}
with open(gold_test_path) as fg:
    for line in fg:
        line = line.rstrip('\r\n').split('\t')
        if line[0] not in gold_aspect_cats:
            gold_aspect_cats[line[0]] = {"starts":[], "ends":[], "cats":[], "sents":[]}
        gold_aspect_cats[line[0]]["starts"].append(int(line[3]))
        gold_aspect_cats[line[0]]["ends"].append(int(line[4]))
        gold_aspect_cats[line[0]]["cats"].append(line[1])
        gold_aspect_cats[line[0]]["sents"].append(line[5])

In [59]:
full_match, partial_match, full_cat_match, partial_cat_match = 0, 0, 0, 0
total = 0
fully_matched_pairs = []
partially_matched_pairs = []
with open(pred_test_path) as fp:
    for line in fp:    
        total += 1
        line = line.rstrip('\r\n').split('\t')
        start, end = int(line[3]), int(line[4])
        category = line[1]
        doc_gold_aspect_cats = gold_aspect_cats[line[0]]
        if start in doc_gold_aspect_cats["starts"]:
            i = doc_gold_aspect_cats["starts"].index(start)
            if doc_gold_aspect_cats["ends"][i] == end:
                full_match += 1
                if doc_gold_aspect_cats["cats"][i] == category:
                    full_cat_match += 1
                else:
                    partial_cat_match += 1
                fully_matched_pairs.append(
                    (
                        [
                            doc_gold_aspect_cats["starts"][i], 
                            doc_gold_aspect_cats["ends"][i], 
                            doc_gold_aspect_cats["cats"][i],
                            doc_gold_aspect_cats["sents"][i]
                        ],
                        line
                    )
                )
                continue
        for s_pos in doc_gold_aspect_cats["starts"]:
            if start <= s_pos:
                i = doc_gold_aspect_cats["starts"].index(s_pos)
                if doc_gold_aspect_cats["ends"][i] == end:
                    partial_match += 1
                    partially_matched_pairs.append(
                        (
                            [
                                doc_gold_aspect_cats["starts"][i], 
                                doc_gold_aspect_cats["ends"][i], 
                                doc_gold_aspect_cats["cats"][i],
                                doc_gold_aspect_cats["sents"][i]
                            ],
                            line
                        )
                    )
                    if doc_gold_aspect_cats["cats"][i] == category:
                        partial_cat_match += 1
                    continue
                matched = False
                for e_pos in doc_gold_aspect_cats["ends"][i:]:
                    if s_pos <= end <= e_pos:
                        partial_match += 1
                        partially_matched_pairs.append(
                            (
                                [
                                    doc_gold_aspect_cats["starts"][i], 
                                    doc_gold_aspect_cats["ends"][i], 
                                    doc_gold_aspect_cats["cats"][i],
                                    doc_gold_aspect_cats["sents"][i]
                                ],
                                line
                            )
                        )
                        if doc_gold_aspect_cats["cats"][i] == category:
                            partial_cat_match += 1
                        matched = True
                        break
                if matched:
                    break
            if start > s_pos:
                i = doc_gold_aspect_cats["starts"].index(s_pos)
                if start < doc_gold_aspect_cats["ends"][i] <= end:
                    partial_match += 1
                    partially_matched_pairs.append(
                        (
                            [
                                doc_gold_aspect_cats["starts"][i], 
                                doc_gold_aspect_cats["ends"][i], 
                                doc_gold_aspect_cats["cats"][i],
                                doc_gold_aspect_cats["sents"][i]
                            ],
                            line
                        )
                    )
                    if doc_gold_aspect_cats["cats"][i] == category:
                        partial_cat_match += 1
                    break

In [60]:
gold_size = sum([len(gold_aspect_cats[x]["cats"]) for x in gold_aspect_cats])

In [ ]:
Реззультаты для разных

In [61]:
#С tf-idf и w2v dict lbfgs
print(f"""
Full match precision: {full_match / total}
Full match recall: {full_match / gold_size}
Partial match ratio in pred: {(full_match + partial_match)  / total}
Full category accuracy: {full_cat_match / total}
Partial category accuracy: {(full_cat_match + partial_cat_match) / total}
""")


Full match precision: 0.7162534435261708
Full match recall: 0.6554621848739496
Partial match ratio in pred: 0.8282828282828283
Full category accuracy: 0.6483011937557392
Partial category accuracy: 0.7943067033976124



In [6]:
#С tf-idf и w2v lbfgs
print(f"""
Full match precision: {full_match / total}
Full match recall: {full_match / gold_size}
Partial match ratio in pred: {(full_match + partial_match)  / total}
Full category accuracy: {full_cat_match / total}
Partial category accuracy: {(full_cat_match + partial_cat_match) / total}
""")


Full match precision: 0.7906724511930586
Full match recall: 0.6126050420168068
Partial match ratio in pred: 0.8893709327548807
Full category accuracy: 0.7657266811279827
Partial category accuracy: 0.8828633405639913



In [55]:
#С tf-idf и w2v dict l2sgd
print(f"""
Full match precision: {full_match / total}
Full match recall: {full_match / gold_size}
Partial match ratio in pred: {(full_match + partial_match)  / total}
Full category accuracy: {full_cat_match / total}
Partial category accuracy: {(full_cat_match + partial_cat_match) / total}
""")


Full match precision: 0.7145421903052065
Full match recall: 0.66890756302521
Partial match ratio in pred: 0.8276481149012568
Full category accuracy: 0.6490125673249552
Partial category accuracy: 0.7953321364452424



In [12]:
#С tf-idf и w2v l2sgd
print(f"""
Full match precision: {full_match / total}
Full match recall: {full_match / gold_size}
Partial match ratio in pred: {(full_match + partial_match)  / total}
Full category accuracy: {full_cat_match / total}
Partial category accuracy: {(full_cat_match + partial_cat_match) / total}
""")


Full match precision: 0.7986270022883295
Full match recall: 0.5865546218487395
Partial match ratio in pred: 0.8947368421052632
Full category accuracy: 0.7768878718535469
Partial category accuracy: 0.8901601830663616



In [49]:
#С tf-idf и w2v dict ap
#худший
print(f"""
Full match precision: {full_match / total}
Full match recall: {full_match / gold_size}
Partial match ratio in pred: {(full_match + partial_match)  / total}
Full category accuracy: {full_cat_match / total}
Partial category accuracy: {(full_cat_match + partial_cat_match) / total}
""")


Full match precision: 0.6966783216783217
Full match recall: 0.6697478991596638
Partial match ratio in pred: 0.8164335664335665
Full category accuracy: 0.6346153846153846
Partial category accuracy: 0.791958041958042



In [18]:
#С tf-idf и w2v ap
print(f"""
Full match precision: {full_match / total}
Full match recall: {full_match / gold_size}
Partial match ratio in pred: {(full_match + partial_match)  / total}
Full category accuracy: {full_cat_match / total}
Partial category accuracy: {(full_cat_match + partial_cat_match) / total}
""")


Full match precision: 0.7494969818913481
Full match recall: 0.6260504201680672
Partial match ratio in pred: 0.8702213279678068
Full category accuracy: 0.7253521126760564
Partial category accuracy: 0.8661971830985915



In [43]:
#С tf-idf и w2v dict pa
print(f"""
Full match precision: {full_match / total}
Full match recall: {full_match / gold_size}
Partial match ratio in pred: {(full_match + partial_match)  / total}
Full category accuracy: {full_cat_match / total}
Partial category accuracy: {(full_cat_match + partial_cat_match) / total}
""")


Full match precision: 0.6963394342762064
Full match recall: 0.7033613445378152
Partial match ratio in pred: 0.8078202995008319
Full category accuracy: 0.6397670549084858
Partial category accuracy: 0.7861896838602329



In [24]:
#С tf-idf и w2v pa
print(f"""
Full match precision: {full_match / total}
Full match recall: {full_match / gold_size}
Partial match ratio in pred: {(full_match + partial_match)  / total}
Full category accuracy: {full_cat_match / total}
Partial category accuracy: {(full_cat_match + partial_cat_match) / total}
""")


Full match precision: 0.7459867799811143
Full match recall: 0.6638655462184874
Partial match ratio in pred: 0.8574126534466477
Full category accuracy: 0.7186024551463644
Partial category accuracy: 0.8517469310670444



In [37]:
#С tf-idf и w2v arow dict
#худший
print(f"""
Full match precision: {full_match / total}
Full match recall: {full_match / gold_size}
Partial match ratio in pred: {(full_match + partial_match)  / total}
Full category accuracy: {full_cat_match / total}
Partial category accuracy: {(full_cat_match + partial_cat_match) / total}
""")


Full match precision: 0.5694029850746268
Full match recall: 0.6411764705882353
Partial match ratio in pred: 0.7328358208955223
Full category accuracy: 0.508955223880597
Partial category accuracy: 0.6865671641791045



In [31]:
#С tf-idf и w2v arow
#худший
print(f"""
Full match precision: {full_match / total}
Full match recall: {full_match / gold_size}
Partial match ratio in pred: {(full_match + partial_match)  / total}
Full category accuracy: {full_cat_match / total}
Partial category accuracy: {(full_cat_match + partial_cat_match) / total}
""")


Full match precision: 0.6182608695652174
Full match recall: 0.5974789915966386
Partial match ratio in pred: 0.7930434782608695
Full category accuracy: 0.5886956521739131
Partial category accuracy: 0.7547826086956522



In [28]:
#Без всего
print(f"""
Full match precision: {full_match / total}
Full match recall: {full_match / gold_size}
Partial match ratio in pred: {(full_match + partial_match)  / total}
Full category accuracy: {full_cat_match / total}
Partial category accuracy: {(full_cat_match + partial_cat_match) / total}
""")



Full match precision: 0.7798066595059077
Full match recall: 0.6100840336134454
Partial match ratio in pred: 0.8872180451127819
Full category accuracy: 0.7604726100966702
Partial category accuracy: 0.8796992481203008

